In [ ]:
import torch
from torch import nn
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(42)

In [ ]:
# 1. carregar dataset real
data = load_diabetes()
X = data.data      # (n_samples, n_features)
y = data.target    # progressão da doença

# 2. separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 3. padronizar features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. converter pra tensor
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32)

input_dim = X_train_t.shape[1]
print("Formato treino:", X_train_t.shape, y_train_t.shape)

In [ ]:
model = nn.Linear(input_dim, 1) # modelo de regressão sem camadas ocultas
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

num_epochs = 10000
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    # forward
    y_pred = model(X_train_t).squeeze(1)
    loss = loss_fn(y_pred, y_train_t)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # salvar loss de treino
    train_losses.append(loss.item())

    # loss de teste
    with torch.no_grad():
        y_test_pred = model(X_test_t).squeeze(1)
        test_loss = loss_fn(y_test_pred, y_test_t).item()
        test_losses.append(test_loss)

    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1:03d} | train loss: {loss.item():.4f} | test loss: {test_loss:.4f}")

In [ ]:
plt.figure(figsize=(7,4))
plt.plot(train_losses, label="Treino")
plt.plot(test_losses, label="Teste")
plt.title("Evolução da loss (MSE) - Rede neural 1 camada")
plt.xlabel("Época")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
with torch.no_grad():
    y_pred_nn = model(X_test_t).squeeze(1).numpy()

mse_nn = mean_squared_error(y_test, y_pred_nn)
rmse_nn = np.sqrt(mse_nn)
mae_nn = mean_absolute_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)

print("===== Rede Neural (1 camada linear) =====")
print(f"MSE :  {mse_nn:.4f}")
print(f"RMSE:  {rmse_nn:.4f}")
print(f"MAE :  {mae_nn:.4f}")
print(f"R²  :  {r2_nn:.4f}")

In [ ]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_pred_skl = linreg.predict(X_test)

mse_skl = mean_squared_error(y_test, y_pred_skl)
rmse_skl = np.sqrt(mse_skl)
mae_skl = mean_absolute_error(y_test, y_pred_skl)
r2_skl = r2_score(y_test, y_pred_skl)

print("===== Regressão Linear (sklearn) =====")
print(f"MSE :  {mse_skl:.4f}")
print(f"RMSE:  {rmse_skl:.4f}")
print(f"MAE :  {mae_skl:.4f}")
print(f"R²  :  {r2_skl:.4f}")

In [ ]:
print("\nPesos da rede neural (PyTorch):")
print(model.weight.data.numpy())
print("Intercepto da rede:", model.bias.data.numpy())

print("\nPesos da regressão linear (sklearn):")
print(linreg.coef_)
print("Intercepto da regressão linear:", linreg.intercept_)